In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/working/augmented_dataset"
batch_size = 64
num_epochs = 20
learning_rate = 0.001
num_classes = len(os.listdir(data_dir))


In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)


In [ ]:
class TamilLetterCNN(nn.Module):
    def __init__(self, num_classes):
        super(TamilLetterCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = TamilLetterCNN(num_classes).to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
        total += inputs.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")

print("Training complete!")


In [ ]:
torch.save(model.state_dict(), "/kaggle/working/tamil_cnn.pth")

In [ ]:
data_dir = "/kaggle/working/augmented_dataset"
class_names = sorted(os.listdir(data_dir))

In [ ]:
infer_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
model = TamilLetterCNN(num_classes=len(class_names)).to(device)
model.load_state_dict(torch.load("/kaggle/working/tamil_cnn.pth", map_location=device))
model.eval()


In [ ]:
def predict_image(image_path, model, class_names, device):
    img = Image.open(image_path).convert("RGB")
    img = infer_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]

test_img_path = "/kaggle/input/tamil-letter/Thamizhi_Dataset_Augmented/test/க/img_0.png"
predicted_letter = predict_image(test_img_path, model, class_names, device)
print(f"Predicted Tamil letter: {predicted_letter}")
